|      |      | 
| ---- | ---- | 
| Journal:|Sustainability (ISSN 2071-1050)|
|Manuscript ID:| sustainability-1485080|
|Type:|Article|
|Title:|NEVs Supply Chain Coordination with Financial Constraint and Demand Uncertainty|
|manuscripts:|P.5 - figure 2. Data generation code|
|Code authors:|Yongjian Li$^*$ |
|Last modify:|2021-01-11|
|version:|Python 3.6.5, sympy >= 1.3|

In [1]:
from sympy import *
from sympy.stats import *
import numpy as np
import pandas as pd
# init_printing(use_unicode=True)
import sympy
sympy.__version__

'1.9'

In [2]:
p, w, c, s, B, b, r, alpha, theta, q, a, x, r, mu, sigma, y = symbols(
    "p,w,c,s,B,b,r,alpha,theta,q,a,x,r,mu,sigma,y")

>  Suppose $x$ is a Normal distribution with parameters $\mu = 100, \sigma = 100$

In [3]:
# Initial parameter values
parameter = {p: 10, c: 4, s: 3, w: 6, B: 100, r: 0.05,
             mu: 100, sigma: 100}

In [4]:
X = Normal(x, mu, sigma).subs(parameter)

(see manuscript P(5) figure 1.)
\begin{align}
&F^{-1}(\alpha) \\
&q_{t}=\frac{(p-s) F^{-1}(\alpha)+B(1+r)}{w(1+r)-s} \\
&q_{s}=F^{-1}\left(\frac{p-c-w r}{p-s}\right)
\end{align}


- exp1: $F(x) = \alpha$

In [5]:
exp1 = Eq(cdf(X)(x), alpha)
exp1

Eq(erf(sqrt(2)*(x - 100)/200)/2 + 1/2, alpha)

- inverse exp1: $F^{-1}(\alpha) = x ,\quad \{0\leqslant \alpha \leqslant 1\}$

In [6]:
i_cdf = solve(exp1, x)[0]
i_cdf

100*sqrt(2)*erfinv(2*alpha - 1) + 100

In [7]:
# Convert a SymPy expression into a function that allows for fast numeric evaluation.
f = lambdify(alpha, i_cdf, modules=['numpy', 'sympy'])

x = [i/1000 for i in range(1, 1000)]

dt = {
    "x_0": x,
}

df = pd.DataFrame(dt)
df['f(x)'] = df['x_0'].map(f)
df['f(x)'] = df['f(x)'].map(int)
df1 = df.drop_duplicates(subset=['f(x)']).reset_index(drop=True)
df1


,x_0,f(x)
0,0.001,-209
1,0.002,-187
2,0.003,-174
3,0.004,-165
4,0.005,-157
...,...,...
422,0.995,357
423,0.996,365
424,0.997,374
425,0.998,387


In [8]:
df1.query("`f(x)`==0")

,x_0,f(x)
113,0.157,0


In [9]:
# df1.to_csv("inverse(CDF)",index=None,sep='\t')

$$q_{t}=\frac{(p-s) F^{-1}(\alpha)+B(1+r)}{w(1+r)-s} $$

In [10]:

qt_f = ((p-s) * a + B*(1+r))/(w*(1+r)-s)
qt_f = qt_f.subs(parameter)

f2 = lambdify(a, qt_f, modules=['numpy', 'sympy'])

qt_values = f2(df['f(x)'])
qt_values = qt_values.to_numpy()

In [11]:

dt = {
    "x_0": x,
    "f(x)": qt_values
}
df_qt = pd.DataFrame(dt)
df_qt['f(x)'] = df_qt['f(x)'].map(int)
df_qt = df_qt.drop_duplicates(subset=['f(x)']).reset_index(drop=True)
df_qt = df_qt.iloc[::10].reset_index(drop=True)
df_qt

,x_0,f(x)
0,0.001,-411
1,0.011,-241
2,0.021,-186
3,0.031,-150
4,0.041,-123
5,0.051,-101
6,0.062,-80
7,0.075,-59
8,0.091,-38
9,0.108,-16


$$q_{s}=F^{-1}\left(\frac{p-c-w r}{p-s}\right)$$

In [12]:
alpha_value = (p-c - w*r)/(p - s)
alpha_value = alpha_value.subs(parameter)
alpha_value

0.814285714285714

In [13]:
# Calculate  qs
qs = int(f(alpha_value))
qs

189

In [14]:
# add point (0,189)
value = solve(Eq(qt_f, qs), a)[0]
d1 = {
    "x_0": [cdf(X)(value).evalf()],
    "f(x)": [qs]
}
df_qt = df_qt.append(pd.DataFrame(d1))


value = solve(Eq(qt_f, 0), a)[0]
d1 = {
    "x_0": [cdf(X)(value).evalf()],
    "f(x)": [0]
}

df_qt = df_qt.append(pd.DataFrame(d1))
df_qt = df_qt.drop_duplicates(subset=['x_0'])
df_qt = df_qt.sort_values(by=['x_0']).reset_index(drop=True)
df_qt

,x_0,f(x)
0,0.001,-411
1,0.011,-241
2,0.021,-186
3,0.031,-150
4,0.041,-123
5,0.051,-101
6,0.062,-80
7,0.075,-59
8,0.091,-38
9,0.108,-16


In [15]:
df_qt1 = df_qt.query("@qs >=`f(x)` >= 0").reset_index(drop=True)
df_qt1

,x_0,f(x)
0,0.125071935637150,0
1,0.128,4
2,0.15,25
3,0.177,46
4,0.204,67
5,0.233,89
6,0.265,110
7,0.299,131
8,0.334,152
9,0.371,173


# 计算 利润
$\pi^\star(q^c) = q(p-c)-(p-s) \int_{0}^{q} F(x) d x-(w q-B)^{+} r$

In [16]:
p, w, c, s, B, b, r, alpha, theta, q, a, x, r, mu, sigma, y = symbols(
    "p,w,c,s,B,b,r,alpha,theta,q,a,x,r,mu,sigma,y")

In [17]:
pi_qc = q*(p-c) - (p-s)*integrate(cdf(X)(x), (x, 0, q)) - Max(0,w*q-B)*r
pi_qc = pi_qc.subs(parameter)
pi_qc

-7*q*erf(sqrt(2)*q/200 - sqrt(2)/2)/2 + 5*q/2 - 350*sqrt(2)*exp(-1/2)*exp(q/100)*exp(-q**2/20000)/sqrt(pi) + 350*erf(sqrt(2)*q/200 - sqrt(2)/2) - 0.05*Max(0, 6*q - 100) + 350*sqrt(2)*exp(-1/2)/sqrt(pi) + 350*erf(sqrt(2)/2)

In [18]:
pi_f = lambdify(q,pi_qc, modules=['numpy', 'sympy'])

In [19]:
df_qt['f(x)'] = df_qt['f(x)'].map(pi_f)
df_qt

,x_0,f(x)
0,0.001,-2407.67919121971
1,0.011,-1387.73751649205
2,0.021,-1058.11405071517
3,0.031,-843.082066614009
4,0.041,-682.819035868065
5,0.051,-553.465288629878
6,0.062,-431.672079316959
7,0.075,-312.336046187489
8,0.091,-196.498621869559
9,0.108,-80.2833271144604


In [26]:
df_qt.to_csv("piqc",index=None,sep='\t')

In [23]:
df_qt.query("0.40>=`x_0`>=0.12").to_csv("piqc_in",index=None,sep='\t')

- some help fun() note here.

In [5]:
## note here
## Given a value, calculate the cumulative probability
value = 0
cdf(X)(value).evalf()
## Given a value, calculate the cumulative probability
value = 189.380
P(X < value).evalf(subs=parameter)